In [12]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

train_files, train_targets = load_dataset('./dogs/train')
valid_files, valid_targets = load_dataset('./dogs/valid')
test_files, test_targets = load_dataset('./dogs/test')

dog_names = [item[20:-1] for item in sorted(glob("./dogs/train/*/"))]

# Let's check the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [13]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [14]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:05<00:00, 158.79it/s]


In [15]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(224, 224, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dense(133, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 16)      432       
_________________________________________________________________
batch_normalization_8 (Batch (None, 224, 224, 16)      48        
_________________________________________________________________
activation_8 (Activation)    (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 32)        4608      
_________________________________________________________________
batch_normalization_9 (Batch (None, 56, 56, 32)       

In [16]:
from keras.callbacks import ModelCheckpoint  

EPOCHS = 10
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=EPOCHS, batch_size=32, callbacks=[checkpointer], verbose=1)

Epoch 1/10
209/209 [==============================] - ETA: 0s - loss: 4.9020 - accuracy: 0.0169
Epoch 00001: val_loss improved from inf to 5.54875, saving model to saved_models/weights.best.from_scratch.hdf5
209/209 [==============================] - 52s 251ms/step - loss: 4.9020 - accuracy: 0.0169 - val_loss: 5.5487 - val_accuracy: 0.0180
Epoch 2/10
209/209 [==============================] - ETA: 0s - loss: 4.4717 - accuracy: 0.0464
Epoch 00002: val_loss improved from 5.54875 to 4.66686, saving model to saved_models/weights.best.from_scratch.hdf5
209/209 [==============================] - 53s 252ms/step - loss: 4.4717 - accuracy: 0.0464 - val_loss: 4.6669 - val_accuracy: 0.0263
Epoch 3/10
209/209 [==============================] - ETA: 0s - loss: 4.2053 - accuracy: 0.0638
Epoch 00003: val_loss did not improve from 4.66686
209/209 [==============================] - 52s 250ms/step - loss: 4.2053 - accuracy: 0.0638 - val_loss: 4.7066 - val_accuracy: 0.0216
Epoch 4/10
209/209 [===========

In [18]:
# model.load_weights('saved_models/weights.best.from_scratch.hdf5')
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 11.3636%


In [19]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

def extract_VGG19(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_vgg19(tensors)
    return VGG19(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

def extract_Resnet50(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_resnet50(tensors)
    return ResNet50(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

In [20]:
train_vgg19 = extract_VGG19(train_files)
valid_vgg19 = extract_VGG19(valid_files)
test_vgg19 = extract_VGG19(test_files)
print("VGG19 shape", train_vgg19.shape[1:])

train_resnet50 = extract_Resnet50(train_files)
valid_resnet50 = extract_Resnet50(valid_files)
test_resnet50 = extract_Resnet50(test_files)
print("Resnet50 shape", train_resnet50.shape[1:])

100%|██████████| 6680/6680 [00:47<00:00, 141.74it/s]


80142336/80134624 [==============================] - 7s 0us/step


  0%|          | 25/6680 [00:00<00:27, 243.92it/s]

VGG19 shape (7, 7, 512)


100%|██████████| 6680/6680 [00:47<00:00, 141.21it/s]


94773248/94765736 [==============================] - 1s 0us/step


100%|██████████| 836/836 [00:05<00:00, 159.19it/s]


Resnet50 shape (7, 7, 2048)


In [21]:
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def input_branch(input_shape=None):
    
    size = int(input_shape[2] / 4)
    
    branch_input = Input(shape=input_shape)
    branch = GlobalAveragePooling2D()(branch_input)
    branch = Dense(size, use_bias=False, kernel_initializer='uniform')(branch)
    branch = BatchNormalization()(branch)
    branch = Activation("relu")(branch)
    return branch, branch_input

vgg19_branch, vgg19_input = input_branch(input_shape=(7, 7, 512))
resnet50_branch, resnet50_input = input_branch(input_shape=(1, 1, 2048))
concatenate_branches = Concatenate()([vgg19_branch, resnet50_branch])
net = Dropout(0.3)(concatenate_branches)
net = Dense(640, use_bias=False, kernel_initializer='uniform')(net)
net = BatchNormalization()(net)
net = Activation("relu")(net)
net = Dropout(0.3)(net)
net = Dense(133, kernel_initializer='uniform', activation="softmax")(net)

model = Model(inputs=[vgg19_input, resnet50_input], outputs=[net])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 7, 7, 512)]  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1, 1, 2048)] 0                                            
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 512)          0           input_7[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           input_8[0][0]                    
_______________________________________________________________________________________

In [24]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/bestmodel.hdf5', 
                               verbose=1, save_best_only=True)
model.fit([train_vgg19, train_resnet50], train_targets, 
          validation_data=([valid_vgg19, valid_resnet50], valid_targets),
          epochs=10, batch_size=4, callbacks=[checkpointer], verbose=1)

Epoch 1/10
1665/1670 [============================>.] - ETA: 0s - loss: 2.5974 - accuracy: 0.3757WARNING:tensorflow:Model was constructed with shape (None, 1, 1, 2048) for input Tensor("input_8:0", shape=(None, 1, 1, 2048), dtype=float32), but it was called on an input with incompatible shape (None, 7, 7, 2048).

Epoch 00001: val_loss improved from inf to 0.98278, saving model to saved_models/bestmodel.hdf5
1670/1670 [==============================] - 7s 4ms/step - loss: 2.5946 - accuracy: 0.3757 - val_loss: 0.9828 - val_accuracy: 0.6982
Epoch 2/10
1665/1670 [============================>.] - ETA: 0s - loss: 1.5425 - accuracy: 0.5670
Epoch 00002: val_loss improved from 0.98278 to 0.81830, saving model to saved_models/bestmodel.hdf5
1670/1670 [==============================] - 7s 4ms/step - loss: 1.5410 - accuracy: 0.5671 - val_loss: 0.8183 - val_accuracy: 0.7557
Epoch 3/10
1665/1670 [============================>.] - ETA: 0s - loss: 1.3412 - accuracy: 0.6230
Epoch 00003: val_loss impro

In [27]:
# model.load_weights('saved_models/bestmodel.hdf5')

from sklearn.metrics import accuracy_score

predictions = model.predict([test_vgg19, test_resnet50])
breed_predictions = [np.argmax(prediction) for prediction in predictions]
breed_true_labels = [np.argmax(true_label) for true_label in test_targets]
print('Test accuracy: %.4f%%' % (accuracy_score(breed_true_labels, breed_predictions) * 100))

Test accuracy: 80.2632%


In [28]:
model_json = model.to_json()
with open('CNN_dogs_breed_model.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('CNN_dogs_breed_model.h5')
print('Saved model to disk')

Saved model to disk


In [35]:
# 안되잖아
# Test를 해보장
test_filenames = os.listdir('./dogvscat/test1/')
test_df = pd.DataFrame({
    'filename':test_filenames
})
nb_samples = test_df.shape[0]

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df,
    './dogvscat/test1/',
    x_col='filename',
    y_col=None,
    target_size=IMAGE_SIZE,
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)

# 모델 예측
predict = model.predict_generator(test_generator,steps=np.ceil(nb_samples/batch_size))

# 평가 생성
# 확률이 큰 걸로 라벨을 붙여준다
test_df['category'] = np.argmax(predict,axis=-1)

# 레이블 변환
# 평가를 위해서 dog,cat으로 있던걸 다시 1,0으로 변환
print(test_df['category'])
# test_df['category'] = test_df['category'].replace({'dog':1,'cat':0})

# 정답 확인!
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12,24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img('./dogvscat/test1/'+filename,target_size=IMAGE_SIZE)
    plt.subplot(6,3,index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + '{}'.format(reverse_answer[category]) + ')')
plt.tight_layout()
plt.show()

Found 12500 validated image filenames.


AssertionError: in user code:

    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/